# Note



### 1. 일간 vs 주간

### 2. 초깃값 설정
지금은 $ E[\theta_t] + \sigma[\theta_t]$

### 3. EWMA
   $\text{weight} = \frac{2}{ \text{window} + 2 }$
   
### 4. threshold

+ Volume bar 에서 $P[b_t = 1]$ 도 EWMA 를 적용해야하는가?
+ Jupyter에서 Multi Processing이 아예 안되는가?

### TODO
1. $b_t$ 는 batch 연산 가능
2. 데이터를 다 numpy 데이터로 변환해서 처리 후 Dataframe으로



In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from bars import TIB,VIB,DIB

from tqdm import tqdm

import multiprocessing as mp

In [2]:
path = 'MiniKospi/'
list_file = os.listdir(path)
list_file.sort()
data = pd.DataFrame(columns=['close','volume']) 
data.index.names = ['timestamp']
cnt=0
for file in tqdm(list_file) :
    tmp = pd.read_csv(path+file,index_col=0)
    tmp = tmp.drop(columns=['date','time','sell','buy'])
    tmp.index = pd.to_datetime(tmp.index)
    tmp.index.names = ['timestamp']
    data = data.append(tmp)
    cnt+=1
    #if cnt > 3: break;
weekly = [g for n, g in data.groupby(pd.Grouper(level='timestamp', freq='W'))]

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:14<00:00,  1.33s/it]


```python
def TIB(weekly_data,threshold=5000,index='timestamp',column='close'): 
def VIB(weekly_data,threshold=20000,index='timestamp',col_price='close',col_volume='volume'):
def DIB(weekly_data,threshold=200000,index='timestamp',col_price='close',col_volume='volume'):        
```

In [ ]:
#tick = TIB(weekly)
#volume = VIB(weekly)
dollar = DIB(weekly)

DIB start


  3%|██▊                                                                             | 1/29 [02:10<1:01:01, 130.78s/it]

In [ ]:
'''tick.to_csv('tick_t5K_full.csv')
volume.to_csv('volume_t20K_full.csv')'''
dollar.to_csv('dib_t6M_.csv')

In [ ]:
dollar = pd.read_csv('dollar_t6M_full.csv',index_col = 'timestamp')

In [ ]:
print('size of tick ' + str(tick.index.size))
print('size of volume ' + str(volume.index.size))
print('size of dollar ' + str(dollar.index.size))

### Time bar

In [ ]:
logic_time = {
    'close'  : 'last'
    }
time_bar = data.resample('H').apply(logic_time)
display(time_bar.index.size)

### close dist

In [ ]:
fig, ax = plt.subplots()
sns.kdeplot(tick['close'], color="green")
sns.kdeplot(volume['close'],  color="red")
sns.kdeplot(dollar['close'], color="blue")
sns.kdeplot(time_bar['close'], color="black")
plt.show()

### Distribution of imbalance bars

In [ ]:
tick['return'] = tick['close'].pct_change()
volume['return'] = volume['close'].pct_change()
dollar['return'] = dollar['close'].pct_change()

time_bar['return'] = time_bar['close'].pct_change()

tick = tick.dropna()
volume = volume.dropna()
dollar = dollar.dropna()
time_bar = time_bar.dropna()

fig, ax = plt.subplots()
sns.kdeplot(tick['return'], color="green")
sns.kdeplot(volume['return'],  color="red")
sns.kdeplot(dollar['return'], color="blue")
#sns.kdeplot(time_bar['return'], color="black")
plt.show()

In [ ]:
 fig, ax1 = plt.subplots()

#plt.plot(data_day.index, data_day['close'],color='#ff0000')
plt.plot(data.index, data['close'],color='#ff0000')
# df_corr = data_day[data_day.columns[0]]
# print(df_corr.autocorr)

''' ax2 = ax2.bar(tick_temp.index, tick_temp['dummy'],color='#5d8c70')'''
#ax2 = ax2.plot(tick.index, tick['close'],color='#5d8c70')
plt.plot(tick.index, tick['close'],color='green')

plt.plot(volume.index, volume['close'],color='red')
plt.plot(volume.index, volume['close'],color='blue')

fig.set_figwidth(15)
fig.set_figheight(5)
plt.show()

### Auto Correlation

In [ ]:
display(time_bar['return'].autocorr(lag=1))
display(tick['return'].autocorr(lag=1))
display(volume['return'].autocorr(lag=1))
display(dollar['return'].autocorr(lag=1))

display(time_bar['close'].autocorr(lag=1))
display(tick['close'].autocorr(lag=1))
display(volume['close'].autocorr(lag=1))
display(dollar['close'].autocorr(lag=1))

# num of imbalance bar 

In [ ]:
tick['dummy'] = 1
volume['dummy'] = 1
dollar['dummy'] = 1
logic_count = {
    'dummy'  : 'sum'
    }

tick_temp = tick.resample ('W' ).apply(logic_count)
volume_temp = volume.resample ('W' ).apply(logic_count)
dollar_temp = dollar.resample ('W' ).apply(logic_count)

fig, ax1 = plt.subplots()
plt.plot(tick_temp.index, tick_temp['dummy'],color='green')
plt.plot(volume_temp.index, volume_temp['dummy'],color='red')
plt.plot(dollar_temp.index, dollar_temp['dummy'],color='blue')
fig.set_figwidth(15)
fig.set_figheight(5)
plt.show()



### Monthly Variance of imbalance bars

In [ ]:
logic_month = {
    'return'  : 'var'
    }
tick_month = tick.resample ('M' ).apply(logic_month)
volume_month = volume.resample ('M' ).apply(logic_month)
dollar_month = dollar.resample ('M' ).apply(logic_month)

fig, ax1 = plt.subplots()
plt.plot(tick_month.index, tick_month['return'],color='green')
plt.plot(volume_month.index, volume_month['return'],color='red')
plt.plot(dollar_month.index, dollar_month['return'],color='blue')
fig.set_figwidth(15)
fig.set_figheight(5)
plt.show()


### STD

In [ ]:
display(np.std(tick['return']))
display(np.std(volume['return']))
display(np.std(dollar['return']))

In [ ]:
display(np.std(tick['close']))
display(np.std(volume['close']))
display(np.std(dollar['close']))